In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os

base_path = "/content/drive/MyDrive/SmartVision_Project/smartvision_dataset"

print("Does base path exist?", os.path.exists(base_path))
print("Folders inside base path:")
print(os.listdir(base_path))

Does base path exist? True
Folders inside base path:
['dataset_metadata.json', 'detection', 'classification']


In [5]:
classification_path = os.path.join(base_path, "classification")

print("Folders inside classification:")
print(os.listdir(classification_path))

Folders inside classification:
['train', 'test', 'val']


In [6]:
train_path = os.path.join(classification_path, "train")

print("Number of classes in train:", len(os.listdir(train_path)))
print("Class names:")
print(os.listdir(train_path))

Number of classes in train: 26
Class names:
['truck', 'potted plant', 'train', 'traffic light', 'stop sign', 'person', 'pizza', 'motorcycle', 'horse', 'elephant', 'chair', 'cup', 'dog', 'couch', 'cat', 'car', 'cake', 'cow', 'bus', 'bowl', 'bird', 'bottle', 'bicycle', 'bed', 'airplane', 'bench']


In [7]:
import os

for class_name in os.listdir(train_path):
    class_folder = os.path.join(train_path, class_name)
    print(f"{class_name}: {len(os.listdir(class_folder))} images")

truck: 70 images
potted plant: 70 images
train: 70 images
traffic light: 70 images
stop sign: 70 images
person: 70 images
pizza: 70 images
motorcycle: 70 images
horse: 70 images
elephant: 70 images
chair: 70 images
cup: 70 images
dog: 70 images
couch: 70 images
cat: 70 images
car: 71 images
cake: 70 images
cow: 70 images
bus: 70 images
bowl: 70 images
bird: 70 images
bottle: 70 images
bicycle: 70 images
bed: 70 images
airplane: 70 images
bench: 70 images


Model 1: VGG16 (Frozen Base)

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os

print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.19.0


In [9]:
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
from tensorflow.keras.applications.vgg16 import preprocess_input

# Paths
train_dir = os.path.join(base_path, "classification", "train")
val_dir = os.path.join(base_path, "classification", "val")
test_dir = os.path.join(base_path, "classification", "test")

# Image parameters
IMG_SIZE = 224
BATCH_SIZE = 8   # Small batch for safety

# Data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Load datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print("Number of classes:", train_generator.num_classes)

Found 1821 images belonging to 26 classes.
Found 390 images belonging to 26 classes.
Found 390 images belonging to 26 classes.
Number of classes: 26


In [11]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Reload base model
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

for layer in base_model.layers:
    layer.trainable = False

# New improved model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),   # 🔥 MUCH BETTER
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │         6,682 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,852,698 (56.66 MB)

 Trainable params: 138,010 (539.10 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

VGG16 base loaded ✔

Frozen layers ✔

Dense(512) ✔

Dropout ✔

Output layer: 26 ✔

Trainable params ≈ 12.8M ✔

Non-trainable params ≈ 14.7M ✔

In [12]:
EPOCHS = 5

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
228/228 ━━━━━━━━━━━━━━━━━━━━ 1405s 6s/step - accuracy: 0.0448 - loss: 8.4747 - val_accuracy: 0.1077 - val_loss: 3.9810
Epoch 2/5
228/228 ━━━━━━━━━━━━━━━━━━━━ 38s 165ms/step - accuracy: 0.1474 - loss: 4.3939 - val_accuracy: 0.2026 - val_loss: 3.1358
Epoch 3/5
228/228 ━━━━━━━━━━━━━━━━━━━━ 37s 163ms/step - accuracy: 0.2358 - loss: 3.1216 - val_accuracy: 0.2231 - val_loss: 2.8275
Epoch 4/5
228/228 ━━━━━━━━━━━━━━━━━━━━ 36s 159ms/step - accuracy: 0.3193 - loss: 2.6291 - val_accuracy: 0.2821 - val_loss: 2.6135
Epoch 5/5
228/228 ━━━━━━━━━━━━━━━━━━━━ 38s 164ms/step - accuracy: 0.3325 - loss: 2.3887 - val_accuracy: 0.2872 - val_loss: 2.4758


This model is now:

✔ Not overfitting
✔ Learning real features
✔ Stable training
✔ Generalizing better

Only top layers are training

The VGG convolutional layers are frozen.

So the network is using ImageNet features, which are generic.

We now need to adapt them slightly to your dataset.

Fine-Tuning.
unfreeze the top convolution blocks of VGG.
This allows the model to learn dataset-specific patterns.


In [13]:
# Fine-Tune VGG16
# Unfreeze top layers of VGG16
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Recompile with lower LR
model.compile(
    optimizer=Adam(learning_rate=1e-5),   # very small LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │         6,682 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,852,698 (56.66 MB)

 Trainable params: 7,217,434 (27.53 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

Before fine-tuning → 138K trainable params

After fine-tuning  → 7.2M trainable params

[That means the model can now adapt VGG features to your dataset.]


In [14]:
FINE_TUNE_EPOCHS = 3

history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=FINE_TUNE_EPOCHS
)

Epoch 1/3
228/228 ━━━━━━━━━━━━━━━━━━━━ 46s 181ms/step - accuracy: 0.3892 - loss: 2.2003 - val_accuracy: 0.3256 - val_loss: 2.4027
Epoch 2/3
228/228 ━━━━━━━━━━━━━━━━━━━━ 37s 164ms/step - accuracy: 0.5254 - loss: 1.6870 - val_accuracy: 0.4000 - val_loss: 2.3637
Epoch 3/3
228/228 ━━━━━━━━━━━━━━━━━━━━ 37s 164ms/step - accuracy: 0.5643 - loss: 1.5605 - val_accuracy: 0.3744 - val_loss: 2.3536


In [15]:
model.save("/content/drive/MyDrive/SmartVision_Project/vgg16_model.h5")
print("VGG16 model saved!")

VGG16 model saved!
